##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to the Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [2]:
import io
import re
import string
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization


Install and import the Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 6.1 MB/s 


In [4]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [5]:
url = 'https://github.com/eyalmazuz/AttackAttributionDataset/archive/refs/heads/master.zip' 

dataset = tf.keras.utils.get_file('master.zip', url,extract=True, cache_dir='.',cache_subdir='')

1196032/Unknown - 0s 0us/step

In [6]:
# Normalize pixel values between 0 and 1
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 5
seed = 42

train_ds = tf.keras.utils.text_dataset_from_directory(
    'AttackAttributionDataset-master',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = train_ds.class_names
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'AttackAttributionDataset-master',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 238 files belonging to 12 classes.
Using 191 files for training.
Found 238 files belonging to 12 classes.
Using 47 files for validation.


# Text preprocessing
Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. 

In [7]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 100000
sequence_length = 1000

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) classes - [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) and [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [8]:
min_embedding_dim=16
max_embedding_dim=1024


def model_builder(hp):
  model = keras.Sequential()
  model.add(vectorize_layer)
  hp_units_embed = hp.Int('units_emb_1', min_value=min_embedding_dim, max_value=max_embedding_dim, step=32)
  model.add(Embedding(vocab_size,output_dim= hp_units_embed, name="embedding"))
  model.add(GlobalAveragePooling1D())
 
  # # Tune the number of layers. Best to do it manually for now
  # for i in range(hp.Int("num_layers", 1, 3)):
  #     model.add(
  #         layers.Dense(
  #             # Tune number of units separately.
  #             units=hp.Int(f"units_{i}", min_value=382, max_value=512, step=32),
  #             activation=hp.Choice("activation", ["relu", "tanh"]),
  #         )
  #     )


  # Tune the number of units in the first Dense layer + the activation function
  # Choose an optimal value between 32-512
  hp_units_1 = hp.Int('units_dense_1', min_value=200, max_value=1024, step=20)
  hp_units_2 = hp.Int('units_dense_2', min_value=58, max_value=512, step=10)
  hp_units_3 = hp.Int('units_dense_3', min_value=32, max_value=192, step=10)
  hp_drop_1 = hp.Choice('units_drop_1', values=[0.01, 0.05, 0.1,0.15, 0.2, 0.25, 0.3])
  hp_drop_2 = hp.Choice('units_drop_2', values=[0.01, 0.05, 0.1,0.15, 0.2, 0.25, 0.3])


  model.add(keras.layers.Dense(units=hp_units_1, activation= hp.Choice("activation_1", ["relu", "tanh"])))
  model.add(Dense(units=hp_units_2, activation=hp.Choice("activation_2", ["relu", "tanh"])))
  model.add(tf.keras.layers.Dropout(hp_drop_1)) 
  model.add(Dense(units=hp_units_3, activation=hp.Choice("activation_3", ["relu", "tanh"])))
  model.add(tf.keras.layers.Dropout(hp_drop_2))
  model.add(Dense(16, activation=hp.Choice("activation_4", ["relu", "tanh"])))
  model.add(Dense(len(class_names)))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.005,  0.0011, 0.001,0.0009, 0.0005, 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 5e-3, 1.1e-3, 9e-4, 5e-4, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [9]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
stop_early_val_accuracy = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [11]:
tuner.search(train_ds, epochs=50, validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyper parameters for the model are: {best_hps.values}")
print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is {best_hps.get('units_dense_1')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.""")

Trial 30 Complete [00h 00m 34s]
val_accuracy: 0.5106382966041565

Best val_accuracy So Far: 0.6170212626457214
Total elapsed time: 00h 07m 14s
INFO:tensorflow:Oracle triggered exit
Best Hyper parameters for the model are: {'units_emb_1': 240, 'units_dense_1': 240, 'units_dense_2': 388, 'units_dense_3': 172, 'units_drop_1': 0.05, 'units_drop_2': 0.01, 'activation_1': 'tanh', 'activation_2': 'tanh', 'activation_3': 'tanh', 'activation_4': 'tanh', 'learning_rate': 0.0009, 'tuner/epochs': 15, 'tuner/initial_epoch': 5, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is 240 and the optimal learning rate for the optimizer is 0.0009.


## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [12]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
39/39 [==============================] - 1s 16ms/step - loss: 2.4163 - accuracy: 0.2304 - val_loss: 2.4620 - val_accuracy: 0.1915
Epoch 2/50
39/39 [==============================] - 0s 12ms/step - loss: 2.2034 - accuracy: 0.2565 - val_loss: 2.2401 - val_accuracy: 0.2979
Epoch 3/50
39/39 [==============================] - 0s 12ms/step - loss: 1.8116 - accuracy: 0.3560 - val_loss: 2.0791 - val_accuracy: 0.2979
Epoch 4/50
39/39 [==============================] - 0s 12ms/step - loss: 1.4895 - accuracy: 0.5759 - val_loss: 1.7166 - val_accuracy: 0.4681
Epoch 5/50
39/39 [==============================] - 0s 12ms/step - loss: 0.8953 - accuracy: 0.8168 - val_loss: 1.6037 - val_accuracy: 0.5106
Epoch 6/50
39/39 [==============================] - 0s 12ms/step - loss: 0.6612 - accuracy: 0.8901 - val_loss: 1.4711 - val_accuracy: 0.5319
Epoch 7/50
39/39 [==============================] - 0s 11ms/step - loss: 0.5338 - accuracy: 0.9162 - val_loss: 1.7757 - val_accuracy: 0.4468
Epoch 8/50
39

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [13]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_ds, epochs=best_epoch, validation_data=val_ds)

Epoch 1/41
39/39 [==============================] - 1s 15ms/step - loss: 2.4221 - accuracy: 0.2094 - val_loss: 2.4173 - val_accuracy: 0.1915
Epoch 2/41
39/39 [==============================] - 0s 12ms/step - loss: 2.1882 - accuracy: 0.2513 - val_loss: 2.1317 - val_accuracy: 0.2979
Epoch 3/41
39/39 [==============================] - 0s 12ms/step - loss: 1.7271 - accuracy: 0.3979 - val_loss: 2.0542 - val_accuracy: 0.2340
Epoch 4/41
39/39 [==============================] - 0s 12ms/step - loss: 1.4138 - accuracy: 0.5288 - val_loss: 1.7761 - val_accuracy: 0.4255
Epoch 5/41
39/39 [==============================] - 0s 12ms/step - loss: 1.1715 - accuracy: 0.5916 - val_loss: 1.6986 - val_accuracy: 0.4255
Epoch 6/41
39/39 [==============================] - 0s 12ms/step - loss: 0.7467 - accuracy: 0.8743 - val_loss: 1.4675 - val_accuracy: 0.6383
Epoch 7/41
39/39 [==============================] - 0s 12ms/step - loss: 0.6051 - accuracy: 0.8901 - val_loss: 1.6948 - val_accuracy: 0.5106
Epoch 8/41
39

## Model Metrics

In [14]:
# Model Metrics
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import numpy as np

y_labels = val_ds.map(lambda x,y: y)
print(y_labels.unbatch)

y_labels = y_labels.as_numpy_iterator()

## Reduce to 1D Array
y_labels = np.concatenate([y for x, y in val_ds], axis=0)

y_hat_probs = hypermodel.predict(val_ds)

y_hat_classes = y_hat_probs.argmax(axis=-1)


## accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_labels, y_hat_classes)
print('Accuracy: %f' % accuracy)

## precision tp / (tp + fp)
precision = precision_score(y_labels, y_hat_classes, average=None)
print('Precision (None): %f' % precision.sum())
precision = precision_score(y_labels, y_hat_classes, average='micro')
print('Precision (micro): %f' % precision)
precision = precision_score(y_labels, y_hat_classes, average= 'macro')
print('Precision (macro): %f' % precision)
# precision = precision_score(y_labels, y_hat_classes, average= 'samples')
# print('Precision (samples): %f' % precision)
precision = precision_score(y_labels, y_hat_classes, average= 'weighted')
print('Precision (weighted): %f' % precision)

## recall: tp / (tp + fn)
recall = recall_score(y_labels, y_hat_classes, average='micro')
print('Recall (micro): %f' % recall)
recall = recall_score(y_labels, y_hat_classes, average='macro')
print('Recall (macro): %f' % recall)
# recall = recall_score(y_labels, y_hat_classes, average='samples')
# print('Recall (samples): %f' % recall)
recall = recall_score(y_labels, y_hat_classes, average='weighted')
print('Recall (weighted): %f' % recall)

## f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_labels, y_hat_classes, average='micro')
print('F1 score (micro): %f' % f1)
f1 = f1_score(y_labels, y_hat_classes, average='macro')
print('F1 score (macro): %f' % f1)
f1 = f1_score(y_labels, y_hat_classes, average='weighted')
print('F1 score (weighted): %f' % f1)
 
## kappa
kappa = cohen_kappa_score(y_labels, y_hat_classes)
print('Cohens kappa: %f' % kappa)

## ROC AUC
# print(np.sum(y_hat_probs))
# y_hat_probs = y_hat_probs / np.sum(y_hat_probs)
# print(np.sum(y_hat_probs))
# auc = roc_auc_score(y_labels, y_hat_probs, multi_class='ovo')
# print('ROC AUC: %f' % auc)

## confusion matrix
matrix = confusion_matrix(y_labels, y_hat_classes)
print(matrix)

<bound method DatasetV2.unbatch of <MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>>
Accuracy: 0.659574
Precision (None): 8.376190
Precision (micro): 0.659574
Precision (macro): 0.698016
Precision (weighted): 0.744833
Recall (micro): 0.659574
Recall (macro): 0.672619
Recall (weighted): 0.659574
F1 score (micro): 0.659574
F1 score (macro): 0.650938
F1 score (weighted): 0.667827
Cohens kappa: 0.621540
[[1 0 0 0 0 0 0 0 0 0 0 1]
 [0 6 2 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 3 0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 2 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0]
 [1 0 0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 2 2 0 0 0]
 [0 0 0 0 0 0 1 0 2 4 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 3]]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.

## Summary

In this tutorial, you learned how to use the Keras Tuner to tune hyperparameters for a model. To learn more about the Keras Tuner, check out these additional resources:

* [Keras Tuner on the TensorFlow blog](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Keras Tuner website](https://keras-team.github.io/keras-tuner/)

Also check out the [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) in TensorBoard to interactively tune your model hyperparameters.